https://adventofcode.com/2015

In [ ]:
# @title Download Inputs
year = 2015 # @param {type:"integer"}

!pip install -q 'advent-of-code-data[nb]'

import os
os.environ['AOC_SESSION'] = '53616c7465645f5fff2fcc509a3a0f13ebc7e030d9d5819056b812eb14da7dcea31ada25507ca09c5b7f1ba139cfc1cb1659d303ed3e512a0444a2224945055f'

os.system(f'mkdir {year}')
for day in range(1, 26):
    os.system(f'aocd {day} {year} > {year}/day{day:02}.txt')

# Day 1: Not Quite Lisp ⭐⭐

In [ ]:
def find_floor(characters: str) -> int:
    return sum(1 if ch == '(' else -1 for ch in characters)


assert find_floor('(())') == find_floor('()()') == 0
assert find_floor('(((') == find_floor('(()(()(') == 3
assert find_floor('))(((((') == 3
assert find_floor('())') == find_floor('))(') == -1
assert find_floor(')))') == find_floor(')())())') == -3

In [ ]:
def find_position_basement(characters: str) -> int:
    floor = 0
    for ix, ch in enumerate(characters, start=1):
        floor += 1 if ch == '(' else -1
        if floor == -1:
            return ix

assert find_position_basement(')') == 1
assert find_position_basement('()())') == 5

In [ ]:
with open('2015/day01.txt') as file:
    puzzle_input: str = file.read().strip()
    print(find_floor(puzzle_input))
    print(find_position_basement(puzzle_input))

138
1771


# Day 2: I Was Told There Would Be No Math ⭐⭐

In [ ]:
def calculate_sq_ft(dimensions: list[str]) -> int:

    total_sq_ft = 0
    for dimension in dimensions:
        l, w, h = map(float, dimension.split('x'))
        side_a, side_b = sorted((l, w, h))[:2]
        sq_ft = (2 * l * w) + (2 * w * h) + (2 * h * l)
        slack = side_a * side_b
        total_sq_ft += sq_ft + slack

    return total_sq_ft


assert calculate_sq_ft(['2x3x4']) == 58
assert calculate_sq_ft(['1x1x10']) == 43

In [ ]:
def calculate_ribbon_ft(dimensions: list[str]) -> int:

    total_ribbon_ft = 0
    for dimension in dimensions:
        l, w, h = map(float, dimension.split('x'))
        side_a, side_b = sorted((l, w, h))[:2]
        total_ribbon_ft += (side_a * 2) + (side_b * 2) + (l * w * h)

    return total_ribbon_ft


assert calculate_ribbon_ft(['2x3x4']) == 34
assert calculate_ribbon_ft(['1x1x10']) == 14

In [ ]:
with open('2015/day02.txt') as file:
    puzzle_input: list[str] = file.read().strip().split('\n')
    print(calculate_sq_ft(puzzle_input))
    print(calculate_ribbon_ft(puzzle_input))

1606483.0
3842356.0


# Day 5: Doesn't He Have Intern-Elves For This? ⭐

In [ ]:
import re

def count_nice_strings(strings: list[str]) -> int:

    count = 0

    for string in strings:

        vowels = re.findall(r'[aeiou]', string)
        repeated = re.findall(r'(\w)\1+', string)
        is_naughty = any(i in string for i in ['ab', 'cd', 'pq', 'xy'])

        if len(vowels) >= 3 and len(repeated) >= 1 and not is_naughty:
           count += 1

    return count


assert count_nice_strings(['ugknbfddgicrmopn']) == 1
assert count_nice_strings(['aaa']) == 1
assert count_nice_strings(['jchzalrnumimnmhp']) == 0
assert count_nice_strings(['haegwjzuvuyypxyu']) == 0
assert count_nice_strings(['dvszwmarrgswjxmb']) == 0

In [ ]:
with open('2015/day05.txt') as file:
    puzzle_input: list[str] = file.read().strip().split('\n')
    print(count_nice_strings(puzzle_input))

238


# Day 6: Probably a Fire Hazard ⭐⭐

In [ ]:
import numpy as np
import re

def count_lit_lights(instructions: list[str]) -> int:

    lights = np.zeros((1000, 1000), dtype=bool)

    for instruction in instructions:

        (a, b), (c, d) = np.asarray(re.findall('(\d*),(\d*)', instruction),
                                    dtype='u8')

        c, d = int(c + 1), int(d + 1)

        if instruction.startswith('turn on'):
            lights[a:c, b:d] = True

        elif instruction.startswith('turn off'):
            lights[a:c, b:d] = False

        elif instruction.startswith('toggle'):
            lights[a:c, b:d] = np.invert(lights[a:c, b:d])

    return np.sum(lights)


def brightness_lights(instructions: list[str]) -> int:

    lights = np.zeros((1000, 1000))

    for instruction in instructions:

        (a, b), (c, d) = np.asarray(re.findall('(\d*),(\d*)', instruction),
                                    dtype='u8')

        c, d = int(c + 1), int(d + 1)

        if instruction.startswith('turn on'):
            lights[a:c, b:d] += 1

        elif instruction.startswith('turn off'):
            lights[a:c, b:d] = np.clip(lights[a:c, b:d] - 1, 0, None)

        elif instruction.startswith('toggle'):
            lights[a:c, b:d] += 2

    return int(np.sum(lights))

In [ ]:
with open('2015/day06.txt') as file:
    puzzle_input: list[str] = file.read().strip().split('\n')
    print(count_lit_lights(puzzle_input))
    print(brightness_lights(puzzle_input))

377891
14110788


# Day 9: All in a Single Night ⭐⭐

In [24]:
from collections import defaultdict as ddict
import itertools
import re

def find_distance_of_shortest_route(routes: list[str]) -> int:

    cities, distances = set(), ddict(dict)
    for route in routes:
        start, end, distance = re.findall(r'(\w+) to (\w+) = (\d+)', route)[0]
        distances[start][end] = int(distance)
        distances[end][start] = int(distance)
        cities.add(start)
        cities.add(end)

    tsp_distances = ddict(int)
    possible_routes = list(itertools.permutations(cities))
    for route in possible_routes:
        for start, end in zip(route, route[1:]):
            tsp_distances[route] += distances[start][end]

    # return min(tsp_distances.values())
    return max(tsp_distances.values())


example_route = ['London to Dublin = 464',
                 'London to Belfast = 518',
                 'Dublin to Belfast = 141']

# assert find_distance_of_shortest_route(example_route) == 605
assert find_distance_of_shortest_route(example_route) == 982

In [25]:
with open('2015/day09.txt') as file:
    puzzle_input: list[str] = file.read().strip().split('\n')
    print(find_distance_of_shortest_route(puzzle_input))

736


# Day 12: JSAbacusFramework.io ⭐

In [47]:
import re

def sum_of_all_numbers(objects: list[str]) -> int:
    total = 0
    for obj in objects:
        total += sum(map(int, re.findall('-?\d+', obj)))
    return total


assert sum_of_all_numbers(['[1,2,3]']) == sum_of_all_numbers(['{"a":2,"b":4}']) == 6
assert sum_of_all_numbers(['[[[3]]]']) == sum_of_all_numbers(['{"a":{"b":4},"c":-1}']) == 3
assert sum_of_all_numbers(['{"a":[-1,1]}']) == sum_of_all_numbers(['[-1,{"a":1}]']) == 0
assert sum_of_all_numbers(['[]']) == sum_of_all_numbers(['{}']) == 0

In [48]:
with open('2015/day12.txt') as file:
    puzzle_input: list[str] = file.read().strip().split('\n')
    print(sum_of_all_numbers(puzzle_input))

156366


# Day 17: No Such Thing as Too Much ⭐

In [67]:
import itertools

def count_combos_containers(sizes: list[int], eggnog_litres: int) -> int:
    count = 0
    for size in range(1, len(sizes) + 1):
        for combo in itertools.combinations(sizes, size):
            if sum(combo) == eggnog_litres:
                count += 1
    return count


assert count_combos_containers([20, 15, 10, 5, 5], 25) == 4

In [68]:
with open('2015/day17.txt') as file:
    puzzle_input: list[int] = list(map(int, file.read().strip().split('\n')))
    print(count_combos_containers(puzzle_input, 150))

1638
